# Session #2  Homework

This notebook represent the #2 Homework of ML-Zoomcamp 

### Dataset
In this homework, we will use the California Housing Prices from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

The goal of this homework is to create a regression model for predicting housing prices (column `'median_house_value'`).

### Features

For the rest of the homework, you'll need to use only these columns:

* `'latitude'`,
* `'longitude'`,
* `'housing_median_age'`,
* `'total_rooms'`,
* `'total_bedrooms'`,
* `'population'`,
* `'households'`,
* `'median_income'`,
* `'median_house_value'`

Select only them.

### Import packages

In [14]:
import pandas as pd
import numpy as np

### Get the data

In [15]:

url_data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'
df = pd.read_csv(url_data)

### Inspect the data

In [16]:
df.sample(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
10417,-117.65,33.53,7.0,6814.0,785.0,2175.0,681.0,10.4900,500001.0,<1H OCEAN
14981,-116.99,32.72,15.0,825.0,130.0,334.0,131.0,4.0391,169500.0,<1H OCEAN
3075,-119.23,35.77,36.0,3225.0,635.0,2034.0,593.0,2.4044,72500.0,INLAND
19610,-121.14,37.48,6.0,1772.0,332.0,1011.0,331.0,3.7045,128100.0,INLAND
9453,-123.24,39.81,25.0,1435.0,304.0,746.0,259.0,1.7788,57900.0,INLAND


Select only indicated columns

In [17]:
columns = ['latitude','longitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value']
df = df[columns]

# Question 1
Find a feature with missing values. How many missing values does it have?

In [18]:
df.isnull().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

# Question 2
What's the median (50% percentile) for variable 'population'?

In [19]:
df['population'].median()

1166.0

# Regression

### Split the data

* Shuffle the initial dataset, use seed 42.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Make sure that the target value ('median_house_value') is not in your dataframe.
* Apply the log transformation to the median_house_value variable using the np.log1p() function.

Shuffle the initial dataset, use seed 42.

In [20]:
df = df.sample(frac = 1,random_state=42)


Split your data in train/val/test sets, with 60%/20%/20% distribution.

In [21]:
n = len(df)
n_val = int(n*0.2)
n_test = int(n*0.2)
n_train = n - n_val - n_test

df_val = df[0:n_val]
df_test = df[n_val:n_val+n_test]
df_train = df[n_val+n_test:]

In [22]:
print(n,n_val+n_test+n_train )

20640 20640


Separate target variable ('median_house_value') and remove it 

In [23]:
target_variable = 'median_house_value'
y_train = df_train[target_variable]
del df_train[target_variable]

Apply the log transformation to the median_house_value variable using the np.log1p() function.

In [24]:
y_train = np.log1p(y_train.values)

# Question 3
* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using round(score, 2)
* Which option gives better RMSE?

In [25]:
def linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)


Linear regression (Filling missing data with 0)

In [26]:
def prepare_X(df):
    df = df.fillna(0)
    X = df.values
    return X

X_train = prepare_X(df_train)
w_0, w = linear_regression(X_train , y_train)
y_pred = w_0 + X_train@w
rmse(y_train, y_pred)

0.3369621357945927

Linear regression (Filling missing data with mean)

In [27]:
def prepare_X(df):
    total_bedrooms_mean = df['total_bedrooms'].mean()
    df = df.fillna(10000000)
    X = df.values
    return X

X_train = prepare_X(df_train)
w_0, w = linear_regression(X_train , y_train)
y_pred = w_0 + X_train@w
rmse(y_train, y_pred)

0.3369621357945927